<img src='op2-u02.png'/>
<h2><font color='#7F0000'>OP2-08-Banco de Dados</font></h2>

<table width='100%'>
    <tbody>
        <tr>
            <td width='33%' style='text-align: left; background-color: #DDDDDD; vertical-align: top;'>Notebook Anterior<br><a href="OP2-07-Arquivos-parte-2.ipynb">OP2-07-Arquivos-parte-2</a></td>
            <td width='34%' style='text-align: left; background-color: #DDDDDD; vertical-align: top;'>&nbsp;<br/>
            </td>
            <td width='33%'style='text-align: left; background-color: #DDDDDD; vertical-align: top;'>Próximo Notebook<br/><a href="OP2-09-Aplicacao-com-BD.ipynb">OP2-09-Aplicacao-com-BD</a></td>
        </tr>
    </tbody>
</table>

## SQLite

Segundo sua documentação:
    
> <i>"SQLite é uma biblioteca em linguagem C que implementa um mecanismo de banco de dados SQL pequeno, rápido, independente, de alta confiabilidade e recursos completos."</i>
> <br/> Fonte: https://sqlite.org/index.html
    
É, possivelmente, o SGBDR mais usado no mundo, pois está integrado a plataforma Android e pode ser usado em praticamente qualquer sistema operacional, incluindo dispositivos móveis e computadores.

O Python dispõe de um módulo específico que provê integração simples com o SQLite.

In [ ]:
# importação do módulo SQLite3
import sqlite3

### Conexão com banco de dados

<p>O uso de um banco de dados SQLite requer uma <i>conexão</i>, obtida por meio da função <tt>connect()</tt>, a qual retorna um objeto do tipo <tt>Connection</tt>.</p>

<p>Caso o banco de dados SQLite não exista, ele será criado na operação de conexão.</p>

<p>No exemplo que segue, o banco de dados <tt>metereologia.db</tt> será criado na primeira conexão e apenas aberto nas conexões sucessivas.</p>

In [ ]:
# Conexão com banco de dados SQLite
con = sqlite3.connect('db/metereologia.db')
con

<p>A partir da conexão, usualmente cria-se um <i>cursor</i>, um objeto do tipo <tt>Cursor</tt>, para possibilitar a operação interativa, isto é, a execução de comandos SQL no BD.</p>

In [ ]:
# Criação de cursor para operação interativa
cursor = con.cursor()
cursor

### Execução de comandos SQL

A execução de comandos SQL utiliza o método <tt>execute()</tt> do cursor, tomando como parâmetro o comando SQL que se deseja executar.

É conveniente definir strings para os comandos que serão usados, permitindo seu reuso, além de tornar o código mais organizado e legível.

Por meio da operação interativa é possível executar comandos:

- DDL (<i>Data Definition Language</i>) <tt>create</tt>, <tt>alter</tt> e <tt>drop</tt>;
- DML (<i>Data Manipulation Language</i>) <tt>insert</tt>, <tt>update</tt> e <tt>delete</tt>;
- DQL (<i>Data Query Language</i>) <tt>select</tt>; e
- DTL (<i>Data Transaction Language</i>) <tt>commit</tt> e <tt>rollback</tt>.

Desejamos criar o seguinte <i>schema</i> para o banco de dados <tt>metereologia.db</tt>.

<img src="op2-08-metereologia-01.png" alt="BD metereologia"/>

In [ ]:
# SQL DDL (remoção das tabelas) 
# Remova as tabelas do exemplo quando quiser refazer
# este notebook ou desejar modificar sua estrutura.
sql0 = ['DROP TABLE ESTACOES',
        'DROP TABLE PARAMETROS',
        'DROP TABLE DADOS']

# Execução de comando SQL requer um cursor pré-definido.
for cmd in sql0:
    try:
        # o mesmo cursor é usado para executar vários comandos
        cursor.execute(cmd)
    except Exception as exc:
        print(exc)
# SQL DTL (consolidação de operações)
con.commit()

O uso de string multilinha do Python, delimitadas por aspas triplas (simples ou duplas), é bastante conveniente para garantir a legibilidade dos comandos SQL, que podem ser bastante longos.

In [ ]:
# SQL DDL (criação de tabela)
sql1 = '''CREATE TABLE ESTACOES (
    Codigo varchar(8) PRIMARY KEY,
    Nome varchar(40) not null,
    Latitude float not null,
    Longitude float not null,
    Altitude integer not null
)'''
print(sql1)

Por meio do cursor previamente definido com uso da conexão, é possível executar comandos SQL, que serão pré-processados, requerendo sua consolidação (<tt>commit</tt>) ou reversão (<tt>roll-back</tt>) posterior.

In [ ]:
# Execução do comando SQL é feita com cursor
cursor.execute(sql1)

In [ ]:
# SQL DDL (criação de tabela)
sql2 = '''CREATE TABLE PARAMETROS (
    Coluna varchar(15) PRIMARY KEY,
    Descricao varchar(40) not null,
    Unidade varchar(5) not null
)'''
print(sql2)

In [ ]:
# SQL DDL (criação de tabela)
sql3 = '''CREATE TABLE DADOS (
    Codigo varchar(8) not null,
    Data date not null,
    Precip_Tot float null,
    Temp_Max float null,
    Temp_Med float null,
    Temp_Min float null,
    Umid_Relat float null,
    PRIMARY KEY (Codigo, Data), 
    FOREIGN KEY (Codigo) references Estacoes(Codigo)
)'''
print(sql3)

In [ ]:
# Execução do comando SQL é feita com cursor
cursor.execute(sql2)
cursor.execute(sql3)

In [ ]:
# Consolidação das tabelas
con.commit()

A alteração ou remoção de tabelas requer o mesmo procedimento, ou seja, a execução da operação desejadas, sua consolidação ou reversão.

### Consultas

<p>As consultas (ou <i>queries</i>) são executadas por meio do comando SQL <tt>select</tt>, que permite obter dados de uma ou mais tabelas existentes no banco de dados.</p>

In [ ]:
# SQL DQL (consulta a dados)
sql4 = '''SELECT Codigo, Nome, Latitude, Longitude, Altitude 
    FROM ESTACOES'''

In [ ]:
# Realização de consulta também emprega o cursor
cursor.execute(sql4)

<p>Como as operações de consulta não modificam os dados presentes nas tabelas, não é requerida a consolidação após sua execução.</p>
<p>A recuperação dos dados obtidos por uma consulta requer o uso do método <tt>fetchall()</tt> do cursor, que retorna uma lista contendo os registros lidos.</p>
<p>A lista retornada pode ser processada e tratada como desejado ou necessário para a aplicação.</p>

In [ ]:
# Dados da consulta realizada devem ser recuperados
dados = cursor.fetchall()
type(dados)

In [ ]:
# Define função para exibir dados da tabela Estacoes
def lista_estacoes(dados):
    # Exibição dos dados consultados de maneira tabular
    print('| %-8s | %-40s | %-12s | %-12s | %-8s |' % ('Código', 'Nome', 'Latitude', 'Longitude', 'Altitude'))
    print('+-%8s-+-%40s-+-%12s-+-%12s-+-%8s-+' % (8*'-', 40*'-', 12*'-', 12*'-', 8*'-'))
    for registro in dados:
        print('| %-8s | %-40s | %12.6f | %12.6f | %8d |' % registro)
    print('+-%8s-+-%40s-+-%12s-+-%12s-+-%8s-+' % (8*'-', 40*'-', 12*'-', 12*'-', 8*'-'))
    return

In [ ]:
# Mostra dados da tabela estações
lista_estacoes(dados)

> É esperado que não sejam listados dados na primeira vez que este notebook é executado, pois as tabelas recém-criadas estão vazias (nenhuma operação de inserção foi realizada).

### Inclusão de dados

<p>A criação ou inclusão de dados são executadas por meio do comando SQL <tt>insert</tt>, que possibilita adicionar novos registros em <b>uma</b> tabela existente no banco de dados.</p>
<p>Observe o uso de aspas duplas (para definir a string Python) e de aspas simples (para delimitar string no SQL).</p>

In [ ]:
# SQL DDL (inserção/criação de dados)
sql5 = """INSERT INTO ESTACOES
    (Codigo, Nome, Latitude, Longitude, Altitude)
    VALUES
    ('A744', 'BRAGANCA PAULISTA', -22.951944, -46.530556, 891)"""

In [ ]:
# Realização de inserção, como sempre, emprega cursor
cursor.execute(sql5)

<p>Operações de inclusão de dados modificam o conteúdo das tabelas afetadas, portanto requerem sua consolidação (<tt>commit</tt>) ou sua reversão (<tt>roll-back</tt>).</p>

In [ ]:
# Consolida inserção
con.commit()

In [ ]:
# Realização de consulta também emprega cursor
cursor.execute(sql4)
# Dados da consulta realizada devem ser recuperados
dados = cursor.fetchall()
type(dados)

In [ ]:
# Mostra dados da tabela estações
lista_estacoes(dados)

<p>Seguem inserções adicionais de dados. Observe o uso da barra invertida <tt>/</tt> para permitir a divisão de uma string comum em mais de uma linha.</p>

In [ ]:
# Mais inserções de dados
sql6 = "INSERT INTO ESTACOES(Codigo, Nome, Latitude, Longitude, Altitude) "\
        "VALUES('A706', 'CAMPOS DO JORDAO', -22.7502777, -45.6038888, 891)"
sql7 = "INSERT INTO ESTACOES(Codigo, Nome, Latitude, Longitude, Altitude) "\
        "VALUES('A701', 'SAO PAULO - MIRANTE', -23.496294, 46.620088, 786)"

In [ ]:
# Realização de inserção, como sempre, emprega cursor
cursor.execute(sql6)
cursor.execute(sql7)
# Consolida inserção
con.commit()

In [ ]:
# Realização de consulta também emprega cursor
cursor.execute(sql4)
# Dados da consulta realizada devem ser recuperados
dados = cursor.fetchall()
# Mostra dados da tabela estações
lista_estacoes(dados)

In [ ]:
parametros = [
    ('Latitude','Latitude da estação','˚'),
    ('Longitude','Longitude da estação','˚'),
    ('Altitude','Altitude da estação','m'),
    ('Precip_Tot','Precipitação total','mm'),
    ('Temp_Max','Temperatura máxima','C'),
    ('Temp_Med','Temperatura média','C'),
    ('Temp_Min','Temperatura mínima','C'),
]

# Laço de repetição
for p in parametros:
    sql8 = f"INSERT INTO PARAMETROS(Coluna, Descricao, Unidade) "\
           f"VALUES('{p[0]}', '{p[1]}', '{p[2]}')"
    print(sql8)
    cursor.execute(sql8)
    
# Consolidação das inserções
con.commit()

In [ ]:
# SQL DQL (consulta a dados)
sql8 = '''SELECT Coluna, Descricao, Unidade FROM PARAMETROS ORDER BY Coluna'''

In [ ]:
# Realização de consulta emprega o cursor
cursor.execute(sql8)

In [ ]:
# Dados da consulta realizada devem ser recuperados
dados = cursor.fetchall()
type(dados)

In [ ]:
# Define função para exibir dados da tabela Parametros
def lista_parametros(dados):
    # Exibição dos dados consultados de maneira tabular
    print('| %-15s | %-40s | %-7s |' % ('Coluna', 'Descricao', 'Unidade'))
    print('+-%15s-+-%40s-+-%7s-+' % (15*'-', 40*'-', 7*'-'))
    for registro in dados:
        print('| %-15s | %-40s | %-7s |' % registro)
    print('+-%15s-+-%40s-+-%7s-+' % (15*'-', 40*'-', 7*'-'))
    return

In [ ]:
# Mostra dados da tabela Parametros
lista_parametros(dados)

A visualização de dados em bancos de dados SQLite também pode ser feita por meio de programas específicos, como o *DB Browser for SQLite*.

https://sqlitebrowser.org/

### Desconexão

<p>Depois de utilizar o banco de dados é necessário encerrar a conexão, o que também encerra automaticamente os cursores abertos,  descarta dados de consultas que não tenham sido recuperados e reverta operações pendentes de consolidação.</p>
<p>Desta maneira, é necessário atenção na desconexão para consolidar operações pendentes.</p>

In [ ]:
# Fecha a conexão
cursor.close() # opcional
con.close()

### Inserção de múltiplos dados

<p>É bastante comum que aplicações necessitem importar dados produzidos por outras aplicações. O uso de arquivos CSV nestas situações é frequente como formato dos dados transferidos entre aplicações diferentes.</p>
<p>No exemplo que segue, dados do arquivo <tt>metereologia.db</tt> são importados e carregados numa tabela do SQLite.</p>
<p>Os dados deste arquivos são públicos e foram obtidos do site:<br/> <a href='https://bdmep.inmet.gov.br/'>https://bdmep.inmet.gov.br/</a></p> 

In [ ]:
# Obtenção de conexão
con = sqlite3.connect('db/metereologia.db')
# Criação do cursor
cursor = con.cursor()

In [ ]:
# importação de módulo para leitura de CSV
from csv import reader

In [ ]:
# Definição de comando SQL parametrizável
sql9 = '''INSERT INTO Dados 
    (Codigo,Data,Precip_Tot,Temp_Max,Temp_Med,Temp_Min,Umid_Relat)
    values (?, ?, ?, ?, ?, ?, ?)'''

In [ ]:
# Funcão que efetua leitura do arquivo CSV inserindo
def inserir_dados(nome_arquivo, estacao):
    with open(nome_arquivo, 'r') as file_reader:
        # Usa file handler para criação de um leitor de CSV
        csv_reader = reader(file_reader, delimiter=';')
        # Usa objeto reader para criar lista de listas com list()
        list_of_rows = list(csv_reader)
        list_of_rows.pop(0) # remove o cabeçalho
        # Percorre lista de listas, efetuando uma inserçao
        for row in list_of_rows:
            print(row)
            # Executa instrução SQL inserindo parâmetros
            cursor.execute(sql9, row)
    # Consolida transação
    cursor.connection.commit()
    return

In [ ]:
# Inserção de dados para as estações A701 e A744
inserir_dados('arquivos/dados_A701_D_2021-01-01_2021-03-06.csv', 'A701')
inserir_dados('arquivos/dados_A744_D_2021-01-01_2021-03-06.csv', 'A744')

In [ ]:
# SQL DQL (consulta a dados)
sql10 = 'SELECT * FROM DADOS'
# Execução do comando SQL
cursor.execute(sql10)
# Recuperação dos dados
dados = cursor.fetchall()

In [ ]:
dados

In [ ]:
# Define função para exibir dados da tabela Dados
def lista_dados(dados):
    # Exibição dos dados consultados de maneira tabular
    print('| %-8s | %-10s | %-10s | %-10s | %-10s | %-10s | %-10s |' %
          ('Estacao', 'Data', 'Precip', 'TMax', 'TMed', 'TMin', 'Umid'))
    print('+-%8s-+-%10s-+-%10s-+-%10s-+-%10s-+-%10s-+-%10s-+' % 
          (8*'-', 10*'-', 10*'-', 10*'-', 10*'-', 10*'-', 10*'-'))
    for registro in dados:
        print('| %-8s | %-10s | %10s | %10s | %10s | %10s | %10s |' % 
              registro)
    print('+-%8s-+-%10s-+-%10s-+-%10s-+-%10s-+-%10s-+-%10s-+' % 
          (8*'-', 10*'-', 10*'-', 10*'-', 10*'-', 10*'-', 10*'-'))
    return

In [ ]:
# Listagem parcial dos dados (usa fatiamento e soma de listas)
lista_dados(dados[:10]+dados[-10:])

In [ ]:
# Nova consulta com filtragem de dados
sql11 = "SELECT * FROM DADOS WHERE Precip_Tot > 40"
cursor.execute(sql11)
dados_parciais = cursor.fetchall()
lista_dados(dados_parciais)

In [ ]:
# Não esqueça de consolidar as ações realizadas.
# Fechar a conexão encerra os cursores abertos.
cursor.close() # opcional
con.close()

Outras operações SQL podem ser realizadas de maneira idêntica. Assim, podem ser construídas aplicações completas com o suporte de banco de dados SQLite. Outros bancos de dados requerem o uso de pacotes e módulos específicos, com idiomas próprios, mas cujos princípios são semelhantes aos vistos aqui.

### FIM
### <a href="http://github.com/pjandl/opy2">Oficina Python Intermediário</a>